# ISO TC/211 Harmonised Model Dependency Controls

Connect to the EA app and model repository

In [1]:
from Parameters import *
from EAConnect import *
from HM_Controls import *
import sys
import pandas as pd

# Open EA Repository and find Model
eaApp = openEAapp()
eaRepo = openEArepo(eaApp,repo_path)
try:
    cmMod = eaRepo.Models.GetByName(modelName)
    printTS('Model "' + modelName + '" found with PackageGUID ' + cmMod.PackageGUID )
    printTS('Number of main packages: ' + str(cmMod.Packages.Count))
except Exception as e:
    printTS('Model  "' + modelName + '" not found!')
    # closeEA(eaRepo)
    # sys.exit()

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

2025-02-25 12:35:37   Hi EA - are you there? 
2025-02-25 12:35:37   I am here
2025-02-25 12:35:37   Hi EA - Please open this repository: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea
2025-02-25 12:35:38   OK! Repository C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\ISOTC211_HM EditorialVersion.qea is ready!
2025-02-25 12:35:38   Model "Conceptual Models" found with PackageGUID {7B6B28E9-C583-4363-9E9C-F37A37AE06C9}
2025-02-25 12:35:38   Number of main packages: 5


Get the TC 211 main package

In [2]:
tcmName = 'ISO TC211'
try:
    tcMod = cmMod.Packages.GetByName(tcmName)
    printTS('Model "' + tcmName + '" found with PackageGUID ' + tcMod.PackageGUID )
except Exception as e:
    printTS('Model  "' + tcmName + '" not found!')

printTS('Number of main packages: ' + str(tcMod.Packages.Count))

2025-02-25 12:35:40   Model "ISO TC211" found with PackageGUID {CAB2E56D-50FA-4904-A16C-B34D7AE325B6}
2025-02-25 12:35:40   Number of main packages: 59


 # Create DataFrames for elements and properties
 - Loop through the complete model:
    - For each element: add content to rows for ElementID, GUID, main standard, standard and element name
    - For each attribute or association end: add content to rows for elementID, GUID, main standard, standard, attribute/association name and refElementID

In [3]:
def recElements(pck,df):
    for eaEl in pck.Elements:
        # print('Element: ' + el.Name)
        if eaEl.Type.upper() in ["CLASS","INTERFACE", "DATATYPE","ENUMERATION"]:
           df.loc[len(df)] = [eaEl.ElementID, eaEl.ElementGUID, stPck.PackageGUID, stPck.Name,edPck.PackageGUID, edPck.Name,eaEl.Name,eaEl.Type] 

    for p in pck.Packages:
        recElements(p,df)
    return df

dfEl = pd.DataFrame(columns=['ElementID','GUID','msGUID','MainStandard','edGUID', 'Edition', 'ElementName','Type'])
for stPck in tcMod.Packages:
    printTS('Standard: ' + stPck.Name)
    for edPck in stPck.Packages:
        printTS('Edition: ' + edPck.Name)
        dfEl = recElements(edPck,dfEl)


2025-02-25 12:35:54   Standard: Common types
2025-02-25 12:35:54   Standard: Package Context Diagrams
2025-02-25 12:35:54   Standard: ISO 6709 Standard representation of geographic point location by coordinates
2025-02-25 12:35:54   Edition: ISO 6709 Edition 2
2025-02-25 12:35:54   Edition: ISO 6709 Edition 1 (Corrigendum 1)
2025-02-25 12:35:54   Standard: ISO 19101 Reference model
2025-02-25 12:35:54   Edition: ISO 19101-1 Edition 1
2025-02-25 12:35:54   Edition: ISO 19101-2 Edition 1
2025-02-25 12:35:54   Edition: ISO/TS 19101-2 Edition 1
2025-02-25 12:35:54   Standard: ISO 19103 Conceptual schema language
2025-02-25 12:35:54   Edition: ISO 19103 Edition 2
2025-02-25 12:35:55   Edition: ISO 19103 Edition 2 Informative content
2025-02-25 12:35:55   Edition: ISO 19103 Edition 1
2025-02-25 12:35:56   Edition: ISO TS 19103 Edition 1
2025-02-25 12:35:56   Standard: ISO 19104 Terminology
2025-02-25 12:35:56   Edition: ISO 19104 Edition 1
2025-02-25 12:35:56   Standard: ISO 19105 Conformanc

In [ ]:
def recProperties(pck,df):
    for eaEl in pck.Elements:
        # print('Element: ' + el.Name)
        if eaEl.Type.upper() in ["CLASS","INTERFACE", "DATATYPE","ENUMERATION"]:
            for eaProp in eaEl.Attributes:
                df.loc[len(df)] = [eaEl.ElementID, eaEl.ElementGUID, stPck.PackageGUID, stPck.Name,edPck.PackageGUID,edPck.Name, eaProp.Name,'Attribute',eaProp.ClassifierID] 
            # TODO: Navigable assocation ends
             #Loop for connector dependencies
            for eaCon in eaEl.Connectors:
                if eaCon.SupplierID == eaEl.ElementID:
                    cEnd = eaCon.ClientEnd
                    ClassifierID = eaCon.ClientID 
                else:
                    cEnd = eaCon.SupplierEnd
                    ClassifierID = eaCon.SupplierID

                df.loc[len(df)] = [eaEl.ElementID, eaEl.ElementGUID, stPck.PackageGUID, stPck.Name,edPck.PackageGUID,edPck.Name, cEnd.Role,eaCon.Type,ClassifierID] 

    for p in pck.Packages:
        recProperties(p,df)
    return df

dfProp = pd.DataFrame(columns=['ElementID','GUID','msGUID','MainStandard','edGUID', 'Edition', 'PropertyName','Type','refElementID'])
for stPck in tcMod.Packages:
    printTS('Standard: ' + stPck.Name)
    # if stPck.Elements.Count != 0:

    for edPck in stPck.Packages:
        printTS('Edition: ' + edPck.Name)
        dfProp = recProperties(edPck,dfProp)

2025-02-25 12:37:10   Standard: Common types
2025-02-25 12:37:10   Standard: Package Context Diagrams
2025-02-25 12:37:10   Standard: ISO 6709 Standard representation of geographic point location by coordinates
2025-02-25 12:37:10   Edition: ISO 6709 Edition 2
2025-02-25 12:37:10   Edition: ISO 6709 Edition 1 (Corrigendum 1)
2025-02-25 12:37:10   Standard: ISO 19101 Reference model
2025-02-25 12:37:10   Edition: ISO 19101-1 Edition 1
2025-02-25 12:37:11   Edition: ISO 19101-2 Edition 1
2025-02-25 12:37:11   Edition: ISO/TS 19101-2 Edition 1
2025-02-25 12:37:11   Standard: ISO 19103 Conceptual schema language
2025-02-25 12:37:11   Edition: ISO 19103 Edition 2
2025-02-25 12:37:12   Edition: ISO 19103 Edition 2 Informative content
2025-02-25 12:37:13   Edition: ISO 19103 Edition 1
2025-02-25 12:37:15   Edition: ISO TS 19103 Edition 1
2025-02-25 12:37:16   Standard: ISO 19104 Terminology
2025-02-25 12:37:16   Edition: ISO 19104 Edition 1
2025-02-25 12:37:16   Standard: ISO 19105 Conformanc

In [5]:
print(dfProp.head(10))

   ElementID                                    GUID  \
0          9  {375618E4-18C3-46fb-898E-E141DA840492}   
1          9  {375618E4-18C3-46fb-898E-E141DA840492}   
2         11  {D532960E-61DE-4223-87BC-857100E85973}   
3         10  {A57913B0-B61D-4de4-968A-89771001D198}   
4         10  {A57913B0-B61D-4de4-968A-89771001D198}   
5         10  {A57913B0-B61D-4de4-968A-89771001D198}   
6         10  {A57913B0-B61D-4de4-968A-89771001D198}   
7         10  {A57913B0-B61D-4de4-968A-89771001D198}   
8         60  {1B7F996D-1F75-4924-8BBF-AEE755D776EC}   
9         60  {1B7F996D-1F75-4924-8BBF-AEE755D776EC}   

                                   msGUID  \
0  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
1  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
2  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
3  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
4  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
5  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
6  {44346D64-FF7D-40bb-B945-570B5E243B5C}   
7  {44346D64-FF7D-40bb-

# Count references
- per element
- per element and edition
- per element and main standard

In [6]:
# Total number of references to each element

dfElCounts = dfProp.groupby(['refElementID'])['ElementID'].count().reset_index()
dfElCounts.rename(columns={'ElementID': 'TotalReferences'}, inplace=True)
# print(dfElCounts)

result_df = pd.merge(dfEl, dfElCounts, left_on='ElementID', right_on='refElementID', how='left')
result_df.drop(columns=['refElementID'], inplace=True)
result_df.fillna({'TotalReferences':0},inplace=True)
result_df['TotalReferences'] = result_df['TotalReferences'].astype(int)
dfElStat = result_df

# Total number references to per edition package to each element
#Group by edGUID and refElement
dfElCounts = dfProp.groupby(['refElementID','edGUID'])['ElementID'].count().reset_index()
dfElCounts.rename(columns={'ElementID': 'EditionInternalReferences'}, inplace=True)
#Merge on edGUID and refElement
result_df = pd.merge(dfElStat, dfElCounts, left_on=['ElementID','edGUID'], right_on=['refElementID','edGUID'], how='left')
result_df.drop(columns=['refElementID'], inplace=True)
result_df.fillna({'EditionInternalReferences':0},inplace=True)
result_df['EditionInternalReferences'] = result_df['EditionInternalReferences'].astype(int)
dfElStat = result_df

# Total number references to per main standards package to each element
#Group by msGUID and refElement
dfElCounts = dfProp.groupby(['refElementID','msGUID'])['ElementID'].count().reset_index()
dfElCounts.rename(columns={'ElementID': 'StandardInternalReferences'}, inplace=True)
#Merge on edGUID and refElement
result_df = pd.merge(dfElStat, dfElCounts, left_on=['ElementID','msGUID'], right_on=['refElementID','msGUID'], how='left')
result_df.drop(columns=['refElementID'], inplace=True)
result_df.fillna({'StandardInternalReferences':0},inplace=True)
result_df['StandardInternalReferences'] = result_df['StandardInternalReferences'].astype(int)
dfElStat = result_df
dfElStat['ExternalReferences'] = dfElStat['TotalReferences'] - dfElStat['StandardInternalReferences']
# dfElStat.drop(columns=['StandardInternalReferences','EditionInternalReferences'], inplace=True)


In [7]:
# Statistics per element name per main standard (to summarize all references for different versions of the same element)
elStatByName = dfElStat.groupby(['msGUID', 'MainStandard','ElementName']).agg(
    VersionCount=pd.NamedAgg(column='ElementID', aggfunc='count'),
    ExternalReferences=pd.NamedAgg(column='ExternalReferences', aggfunc='sum'),
    TotalReferences=pd.NamedAgg(column='TotalReferences', aggfunc='sum')
).reset_index()



In [8]:
# Add referenced element name, edition etc to prop Dataframe
dfRefEls = dfElStat[['ElementID','ElementName','MainStandard','Edition']].add_prefix('ref')
dfPropWithNames = pd.merge(dfProp, dfRefEls, left_on='refElementID', right_on='refElementID', how='left')

# Add the total number of references for referenced elements
dfRefElStat= elStatByName.add_prefix('count')
dfPropWithNames = pd.merge(dfPropWithNames, dfRefElStat, left_on=['refMainStandard','refElementName'], right_on=['countMainStandard','countElementName'], how='left')
dfPropWithNames.rename(columns={'countExternalReferences': 'refExternalReferences'}, inplace=True)

#Create filtered prop DataFrame
filtered_dfPropWithNames = dfPropWithNames[(dfPropWithNames['MainStandard'] != dfPropWithNames['refMainStandard']) & (dfPropWithNames['refExternalReferences'] >= 5)]


# Export to Excel

In [9]:
file_path = mainFolder + '\\Dependencies.xlsx'
# Export to Excel 
writer = pd.ExcelWriter(file_path)
dfElStatOut = dfElStat.drop(columns=['ElementID','GUID','msGUID','edGUID'])
dfElStatOut.to_excel(writer,'Elements') 
print(f"Exported elements report to file: {file_path}") 

elStatByNameOut = elStatByName.drop(columns=['msGUID'])
elStatByNameOut.to_excel(writer,'Elements by standard') 
print(f"Exported elements report to file: {file_path}") 

print(dfPropWithNames.columns)
dfPropWithNamesOut = dfPropWithNames.drop(columns=['GUID','msGUID','edGUID'])
dfPropWithNamesOut.to_excel(writer,'Properties') 
print(f"Exported properties report to file: {file_path}") 

print(filtered_dfPropWithNames.columns)
filtered_dfPropWithNames = filtered_dfPropWithNames.drop(columns=['GUID','msGUID','edGUID'])
filtered_dfPropWithNames.to_excel(writer,'PropertiesExternal') 
print(f"Exported filtered properties report to file: {file_path}") 


writer.close()


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_22084\3598369286.py:5: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  dfElStatOut.to_excel(writer,'Elements')


Exported elements report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Dependencies.xlsx
Exported elements report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Dependencies.xlsx
Index(['ElementID', 'GUID', 'msGUID', 'MainStandard', 'edGUID', 'Edition',
       'PropertyName', 'Type', 'refElementID', 'refElementName',
       'refMainStandard', 'refEdition', 'countmsGUID', 'countMainStandard',
       'countElementName', 'countVersionCount', 'refExternalReferences',
       'countTotalReferences'],
      dtype='object')


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_22084\3598369286.py:9: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  elStatByNameOut.to_excel(writer,'Elements by standard')
C:\Users\JETKNU\AppData\Local\Temp\ipykernel_22084\3598369286.py:14: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  dfPropWithNamesOut.to_excel(writer,'Properties')


Exported properties report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Dependencies.xlsx
Index(['ElementID', 'GUID', 'msGUID', 'MainStandard', 'edGUID', 'Edition',
       'PropertyName', 'Type', 'refElementID', 'refElementName',
       'refMainStandard', 'refEdition', 'countmsGUID', 'countMainStandard',
       'countElementName', 'countVersionCount', 'refExternalReferences',
       'countTotalReferences'],
      dtype='object')


C:\Users\JETKNU\AppData\Local\Temp\ipykernel_22084\3598369286.py:19: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  filtered_dfPropWithNames.to_excel(writer,'PropertiesExternal')


Exported filtered properties report to file: C:\Data\GitHub\ISO TC211\HMMG\EditorialVersion\Dependencies.xlsx
